# Agent based modelling of social systems

### Modelling social awereness through a cognitive architecture model
Questions/Topics : 
* What is our question(clear formulation)-> Global awereness
* Topology of network
* Network generator (random?)
* Interaction of agents
* Literature
* Assumptions

### Structure
* Output: Plot Magnitude of event vs Importance of agent (color:fraction of agents, prob. to pass threshold)
* Model: 
 * P(k) distribution of #of outward lines
 * Interaction as a weighted relation of importance of agent and magnitude of event)
* Network: 
 * Random neighbour selection
 * % of neighbours interaction in each time step
* Events
 * Prob. distribution of events log(magn)= $\beta$ - $\gamma$log(occurences)
 * Decay as an exponential function
 * Global magnitude state of each event

In [1]:
import numpy as np
import igraph
import matplotlib.pyplot as plt
import random
from matplotlib import animation
from IPython.display import *
from PIL import Image
#%matplotlib inline

In [2]:
class SocialNet:
    def __init__(self, N):
        """
        Initialize the Graph with N number of agents and 
        give them a number of connections(links) from a 
        power law distribution -> scale-free network. Adjust 
        the size of each vertex to represent the number of
        links it has.
        """
        
        self.time=0
        self.N = N        
        self.g = igraph.Graph(self.N ,directed=True)
        self._create_agents()
        self.visual_style = {}
        self.visual_style["edge_arrow_size"] = 0.5
        layout = self.g.layout("lgl")
        self.visual_style["layout"] = layout
        self.visual_style["vertex_size"]=self.g.vs["size"]
        
        # All events stored in this list! First event is the null-event. All agents start with that.
        self.events=[Event(magnitude=0, t=0)]
    
    def _create_agents(self):
        """Sets attributes of vertices in Graph g"""
        i = 0
        
        for v in self.g.vs:
            i+=1
            if i%100 == 0:
                print('Agent',i)
            # Initialize with null-event
            v["event_id"]=0
            v["event_magnitude"]=0
            
            # Generate number of links and format vertex based on that
            links = self.generate_followers() #self.scale_free_distr() 
            v["size"] = 5 + 3*np.log(links)
            v["links"] = links
            
            if links > 5:
                v["type"] = "core"
                v["color"] = "slate blue"
            else:
                v["type"] = "peripheral"
                v["color"] = "SeaGreen2"  
            
            # Create actual links between agents 
            while links:
                source = v.index
                target = np.random.randint(0, len(self.g.vs))
                if not self.g.are_connected(source, target) and target!=source:
                    self.g.add_edge(source,target)
                    links-=1
                #self.g=self.g.simplify()
        w = 0
        for v in self.g.vs:
            w+=1
            if w%10==0:
                print('Betweenness', w)
            v["importance"]= self.g.betweenness(v)
        return
    
    def generate_followers(self):
        chance=np.random.uniform(0,1)
        if chance>10**(-2.5):
            followers=10**( -0.88*np.log10(chance))
        elif chance>10**(-3):
            followers=10**( -2*np.log10(chance) -2.8)
        elif chance>10**(-5):
            followers=10**( -0.2*np.log10(chance)+2.6)
        else:
            followers=len(self.g.vs)-1
        return int(max(np.round(min(followers,len(self.g.vs)-1)),1))
    
    def scale_free_distr(self):
        """Power law distr P(k)=k^-2. In order to 
        limit the number of links possible distr is truncated
        at 0.001
        """
        p = np.random.rand()
        if p <= 0.001:
            return 1
        else:
            return int(np.round(1/np.sqrt(p)))
        
    def update(self):
        """Update all vertices by propagating the information and setting the new state"""
        # In each time-step events happen to agents.
        for v in self.g.vs:
            new_event = Event(magnitude=None, agent_importance=v["importance"], t=self.time)
            # See whether new event is more interesting than old one
            if new_event.magnitude > v["event_magnitude"]:
                self.events.append(new_event)
                # event id is stored in Event object too
                self.events[-1].id = len(self.events)-1
                v["event_id"]=len(self.events)-1
                v["event_magnitude"]=new_event.magnitude    
                v["color"]=new_event.color
        
        # In each time-step agents communicate.
        """ Dynamics: event taken over deterministicly if it is larger event. Important: one-way comparison! """
        for v in self.g.vs:
            for neighbor in v.neighbors(mode=igraph.OUT):
                # only broadcast to a given proportion of neighbors
                if np.random.uniform(0,1)<0.5:
                    if neighbor["event_magnitude"]<v["event_magnitude"]:
                        neighbor["event_magnitude"]=v["event_magnitude"]
                        neighbor["event_id"]=v["event_id"]
                        neighbor["color"]=v["color"]
                # else do nothing    
               
        # At the end of each time-step decay all events and update all vertice event magnitudes
        # Also, calculate the total number of nodes thinking of event i and update event.awareness
        for single_event in self.events:
            single_event.decay()
            num = sum([v==single_event.id for v in self.g.vs["event_id"]])
            single_event.awareness_array.append(num)
            if num >= single_event.awareness:
                single_event.awareness = num
                single_event.final_time = self.time+1 #update final time
            
        for v in self.g.vs:
            v["event_magnitude"]=self.events[v["event_id"]].magnitude
                
        self.time+=1       
   
       
        
    def draw(self):        
        igraph.plot(self.g, "img/temp%i.png"%self.time, **self.visual_style)
        #display(Image(filename='img/temp%i.png'%self.time))
        
    """def displayPlot(self):
        plt.savefig("img/temp_plot.png")
        display(Image(filename='img/temp_plot.png'))"""

class Agent(igraph.Vertex):
    def __init__(self, name, num_links):
        self.attr = self.attributes()
        self.num_links = num_links
        self.visual_style["vertex_size"]=2*self.num_links
        self.visual_style["edge_arrow_size"] = 0.5
        self.event = 0
    def _create_links(self):
        pass
    def set_attributes(self):
        self.attr["size"] = int(self.num_links*2)
        
class Event:
    """ Single event. """
    def __init__(self,magnitude=None, agent_importance=1, t=0):
        self.id = None
        # This part is so that if I don't give any value, then the magnitude is generated
        if magnitude==None:
            self.initial_magnitude = np.random.power(a=0.02)
        else:
            self.initial_magnitude = magnitude
        self.magnitude = self.initial_magnitude
        self.color= "#%06x" % random.randint(0, 0xFFFFFF)
        self.agent_importance = agent_importance # each event contains initial agent importance
        self.awareness = 1 #max nr of agents thinking of event at a single time step
        self.awareness_array = [1]
        self.init_time = t #time of event creation
        self.final_time = t # time of event peak
        
    def decay(self):
        # keep dt=1 for now since we have discrete time-steps
        gamma=0.0005
        dt=1
        self.magnitude=self.magnitude*(1-gamma*dt)

In [3]:
Net=SocialNet(2000)
maxtime=100
for i in range(maxtime):
    Net.update()
    print("updated",i)
    Net.draw()
    print("drawn", i)
    

t=1
fig = plt.figure()
img0 = Image.open('img/temp%i.png'%t)
im=plt.imshow(img0)
def updatefig(fake):
    global t
    img = Image.open('img/temp%i.png'%((t%maxtime)+1))
    im.set_data(img)
    t+=1
    return im
ani = animation.FuncAnimation(fig, updatefig, interval=300)
plt.show()

Agent 100
Agent 200
Agent 300
Agent 400
Agent 500
Agent 600
Agent 700
Agent 800
Agent 900
Agent 1000
Agent 1100
Agent 1200
Agent 1300
Agent 1400
Agent 1500
Agent 1600
Agent 1700
Agent 1800
Agent 1900
Agent 2000
Betweenness 10
Betweenness 20
Betweenness 30
Betweenness 40
Betweenness 50
Betweenness 60
Betweenness 70
Betweenness 80
Betweenness 90
Betweenness 100
Betweenness 110
Betweenness 120
Betweenness 130
Betweenness 140
Betweenness 150
Betweenness 160
Betweenness 170
Betweenness 180
Betweenness 190
Betweenness 200
Betweenness 210
Betweenness 220
Betweenness 230
Betweenness 240
Betweenness 250
Betweenness 260
Betweenness 270
Betweenness 280
Betweenness 290
Betweenness 300
Betweenness 310
Betweenness 320
Betweenness 330
Betweenness 340
Betweenness 350
Betweenness 360
Betweenness 370
Betweenness 380
Betweenness 390
Betweenness 400
Betweenness 410
Betweenness 420
Betweenness 430
Betweenness 440
Betweenness 450
Betweenness 460
Betweenness 470
Betweenness 480
Betweenness 490
Betweenness 50

In [4]:
fraction = np.asarray([e.agent_importance for e in Net.events])
plt.hist(fraction, bins=300)
plt.ylim([0,10])
plt.show()

In [12]:
def analyze_events(events_list):
    magnitude = np.asarray([e.initial_magnitude for e in events_list if e.awareness>2])
    importance = np.asarray([e.agent_importance for e in events_list if e.awareness>2])
    awareness = np.asarray([e.awareness for e in events_list if e.awareness>2])
    colors = awareness/np.max(awareness)
    log_importance = np.log10(importance)
    plt.figure(1)
    plt.scatter(log_importance, magnitude, c=colors, alpha=0.7, s=40) 
    #plt.xscale('log')
    plt.colorbar()
    
    
    data = np.zeros((10,10))
    importance_range = np.linspace(-2,np.max(log_importance),11)
    importance_range = np.round(importance_range, decimals=1)
    print("Importance range", importance_range)
    magnitude_range = np.linspace(0,1.0,11)
    print("Magnitude range", magnitude_range)
    for (x,y),v in np.ndenumerate(data):            
        data[x,y]=np.average([e.awareness for e in events_list if (
                            importance_range[x]<=np.log10(e.agent_importance)<importance_range[x+1] and
                            magnitude_range[y]<=e.initial_magnitude<magnitude_range[y+1])])
        
    
    plt.figure(2)
    plt.imshow(data.T, interpolation='nearest', origin='lower')
    plt.xticks(range(11),importance_range)
    plt.yticks(range(11),magnitude_range)
    plt.show()
    return 

def analyze_events_nontruncated(events_list):
    magnitude = np.asarray([e.initial_magnitude for e in events_list])
    importance = np.asarray([e.agent_importance for e in events_list])
    awareness = np.asarray([e.awareness for e in events_list])
    colors = awareness/np.max(awareness)
    log_importance = np.log10(importance)
    plt.figure(1)
    plt.scatter(log_importance, magnitude, c=colors, alpha=0.7, s=40) 
    #plt.xscale('log')
    plt.colorbar()
    
    
    data = np.zeros((10,10))
    importance_range = np.linspace(-2,np.max(log_importance),11)
    importance_range = np.round(importance_range, decimals=1)
    print("Importance range", importance_range)
    magnitude_range = np.linspace(0,1.0,11)
    print("Magnitude range", magnitude_range)
    for (x,y),v in np.ndenumerate(data):            
        data[x,y]=np.average([e.awareness for e in events_list if (
                            importance_range[x]<=np.log10(e.agent_importance)<importance_range[x+1] and
                            magnitude_range[y]<=e.initial_magnitude<magnitude_range[y+1])])
        
    
    plt.figure(2)
    plt.imshow(data.T, interpolation='nearest', origin='lower')
    plt.xticks(range(11),importance_range)
    plt.yticks(range(11),magnitude_range)
    plt.show()
    return 

def analyze_graph_unimportance(events_list):
    magnitude = np.asarray([e.initial_magnitude for e in events_list if e.awareness>2])
    #importance = np.asarray([e.agent_importance for e in events_list if e.awareness>2])
    awareness = np.asarray([e.awareness for e in events_list if e.awareness>2])
    #colors = awareness/np.max(awareness)
    log_awareness = np.log10(awareness)
    plt.figure(1)
    plt.scatter(log_awareness, magnitude, alpha=0.7, s=40)
    #plt.xscale('log')
    #plt.colorbar()
    
    
    data = np.zeros((10,10))
    awareness_range = np.linspace(-2,np.max(log_awareness),11)
    awareness_range = np.round(awareness_range, decimals=1)
    print("Importance range", awareness_range)
    magnitude_range = np.linspace(0,1.0,11)
    print("Magnitude range", magnitude_range)
    for (x,y),v in np.ndenumerate(data):            
        data[x,y]=np.average([1 for e in events_list if (
                            awareness_range[x]<=np.log10(e.awareness)<awareness_range[x+1] and
                            magnitude_range[y]<=e.initial_magnitude<magnitude_range[y+1])])
        
    
    plt.figure(2)
    plt.imshow(data.T, interpolation='nearest', origin='lower')
    plt.xticks(range(11),awareness_range)
    plt.yticks(range(11),magnitude_range)
    plt.show()
    return 

In [13]:
#plt.hist(np.asarray([e.initial_magnitude for e in Net.events]))
#plt.show()
all_events=Net.events
analyze_graph_unimportance(all_events)

Importance range [-2.  -1.5 -0.9 -0.4  0.1  0.7  1.2  1.7  2.2  2.8  3.3]
Magnitude range [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]


/home/tamas/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/tamas/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
def event_time_evolution(events, total_time):
    for e in events:
        y = np.zeros(total_time)
        y[e.init_time:]=e.awareness_array[1:]
        plt.plot(range(total_time), y)
    plt.show()

In [30]:
event_time_evolution(all_events,maxtime) #[:int(len(all_events)/2)]

In [29]:
import copy
instances=3
network_size=2000
maxtime=60

all_events=[]
for j in range(instances):
    Net=SocialNet(network_size)
    for i in range(maxtime):
        Net.update()
        print("updated", j,i)
        Net.draw()
        print("drawn", j,i)
    all_events.extend(copy.deepcopy(Net.events))
#analyze_events(all_events)

Agent 100
Agent 200
Agent 300
Agent 400
Agent 500
Agent 600
Agent 700
Agent 800
Agent 900
Agent 1000
Agent 1100
Agent 1200
Agent 1300
Agent 1400
Agent 1500
Agent 1600
Agent 1700
Agent 1800
Agent 1900
Agent 2000
Betweenness 10
Betweenness 20
Betweenness 30
Betweenness 40
Betweenness 50
Betweenness 60
Betweenness 70
Betweenness 80
Betweenness 90
Betweenness 100
Betweenness 110
Betweenness 120
Betweenness 130
Betweenness 140
Betweenness 150
Betweenness 160
Betweenness 170
Betweenness 180
Betweenness 190
Betweenness 200
Betweenness 210
Betweenness 220
Betweenness 230
Betweenness 240
Betweenness 250
Betweenness 260
Betweenness 270
Betweenness 280
Betweenness 290
Betweenness 300
Betweenness 310
Betweenness 320
Betweenness 330
Betweenness 340
Betweenness 350
Betweenness 360
Betweenness 370
Betweenness 380
Betweenness 390
Betweenness 400
Betweenness 410
Betweenness 420
Betweenness 430
Betweenness 440
Betweenness 450
Betweenness 460
Betweenness 470
Betweenness 480
Betweenness 490
Betweenness 50

In [ ]:
""" This cell is here so we can easily output plots of our distribution to show in the presentation. """
def generate_followers(follower_list, chance_list):
    chance=np.random.uniform(0,1)
    chance_list.append(chance)
    if chance>10**(-2.5):
        followers=10**( -0.88*np.log10(chance) )
    elif chance>10**(-3):
        followers=10**( -2*np.log10(chance) -2.8)
    elif chance>10**(-5):
        followers=10**( -0.2*np.log10(chance)+2.6 )
    else:
        followers=10**6.6
    follower_list.append(followers)
    return follower_list, chance_list

def generate_followers_scalefree(follower_list, chance_list):
    chance=np.random.uniform(0,1)
    chance_list.append(chance)
    #if chance>10**(-5):
    followers=10**( -0.88*np.log10(chance) )
    """elif chance>10**(-3):
        followers=10**( -2*np.log10(chance) -2.8)
    elif chance>10**(-5):
        followers=10**( -0.2*np.log10(chance)+2.6 )
    else:
        followers=10**6.6"""
    follower_list.append(followers)
    return follower_list, chance_list

x_axis=[]
follower_list=[]
N=int(1e5)
for i in range(N):
    follower_list, x_axis= generate_followers(follower_list,x_axis)
"""fig=plt.figure()
ax = plt.gca()
ax.scatter(x_axis,follower_list)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlim(xmin=1e-4,xmax=1.2)"""


fig=plt.figure()
ax = plt.gca()
ax.scatter(follower_list,x_axis)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylim(ymin=1e-4,ymax=1.2)

In [ ]:
a=np.linspace(0,100,20)
a[1:]